<a href="https://colab.research.google.com/github/gourab98/Data-Science-Course/blob/main/song_classifier_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import glob
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
#!pip install ktrain
#!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1


import ktrain
from ktrain import text
import numpy as np
import glob
import pandas as pd

In [4]:
all_file= glob.glob("./songData/*.xlsx")
len(all_file)

21

# Preprocessing steps

In [5]:
names = []
count = []
clusters = []
data = []

for each in all_file:
    temp = pd.read_excel(each)
    
    
    count.append(len(temp))
    names.append(each.split('/')[1][:-5])
    clusters.append(temp['songType'].iloc[0])
    
    
    data.append(temp)
    
cluster_ids =clusters

In [6]:
temp

,Unnamed: 0,lyrics,songType
0,2,আমি কুকুর হইয়া দেই পাহাড়া দয়াল চান্দের ঘড় বাড়ি...,11
1,3,তুমি এসমে আজ যত তরানে ওলা গাউছেলাম বাবা নুরে ম...,11
2,17,আমি তোর পীড়িতের মরা আমি তোর পীড়িতের মরা বন্ধু ...,11
3,18,মানুষ ধরো মানুষ ভজ শোন বলিরে পাগল মন। মানুষের ...,11
4,19,সাথি পুরা বোতল দে আমারে নেশায় মজে লই… ঐ নেশাতে...,11
...,...,...,...
57,3837,গান গাই আমার মনরে বুঝাই মন থাকে পাগলপারা আর কি...,11
58,4067,মাটিরও পিঞ্জিরার মাঝে বন্দী হইয়া রে কান্দে হাছ...,11
59,4078,বসুন্ধরার বুকে বরষারি ধারা ধারা ভরা হাহাকার (২...,11
60,4079,কেন পিড়িতি বাড়াইলা রে বন্ধু ছেড়ে যাইবা যদি কেম...,11


In [7]:
data = pd.concat(data)
data = data.sample(frac = 1)

data

,Unnamed: 0,lyrics,songType
750,3240,আজি শুভদিনে পিতার ভবনে অমৃত সদনে চলো যাই চলো চ...,17
164,1196,বিশ্ববিদ্যালয়ে সারারাত অবিরাম গোলাগুলি আর পরদ...,13
289,922,মন বলে কাছে তুমি ডাকবে সারাদিন পাশে পাশে থাকবে...,1
7,13,প্রিয় কোথায় তুমি কোন গহনে কোন ধ্রুবলোকে কোন ...,7
226,1391,তুমি আসবে বলে হে প্রিয় নারী এ শহরের চাবি তোমা...,13
...,...,...,...
221,1633,তোমার সোনার থালায় সাজাব আজ দুখের অশ্রুধার। ...,17
71,974,"আকাশে আজ ছড়িয়ে দিলাম প্রিয়। আমার কথার ফুল গো, ...",7
395,2466,"ও বন্ধু লাল গোলাপী, ও বন্ধু লাল গোলাপী কই রইলা...",3
35,113,"আমি তো আজ ভুলে গেছি সবই, ঐ চোখে চেয়ে চেয়ে হয়ে ...",3


### Removing  Stopwords  from body if exists

In [8]:
stopwords = [ 'হাই', 'ফ্রিকুয়েন্সি', 'কিন্তু', 'অপ্রয়োজনীয়', 'শব্দ' ]

In [9]:
data['lyrics'] = data['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [10]:
X_train, X_test, y_train, y_test =  train_test_split(data['lyrics'], data['songType'], test_size=0.33, random_state=42 )

In [11]:
y_train = [str(x) for x in y_train]
y_test = [str(x) for x in y_test]

MODEL_NAME = 'neuralspace-reverie/indic-transformers-bn-distilbert'
t = text.Transformer(MODEL_NAME, maxlen=500)

trn = t.preprocess_train(list(X_train), y_train)
val = t.preprocess_test(list(X_test), y_test)


preprocessing train...
language: bn
train sequence lengths:
	mean : 90
	95percentile : 177
	99percentile : 249


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 92
	95percentile : 184
	99percentile : 273


In [12]:
model = t.get_classifier()

In [13]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=8)

In [16]:
learner.fit_onecycle(1e-5, 8)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/8
344/344 [==============================] - 188s 545ms/step - loss: 1.6341 - accuracy: 0.4822 - val_loss: 1.5813 - val_accuracy: 0.4753
Epoch 2/8
344/344 [==============================] - 187s 543ms/step - loss: 1.4972 - accuracy: 0.5138 - val_loss: 1.4634 - val_accuracy: 0.4827
Epoch 3/8
344/344 [==============================] - 187s 543ms/step - loss: 1.3389 - accuracy: 0.5582 - val_loss: 1.3915 - val_accuracy: 0.5144
Epoch 4/8
344/344 [==============================] - 187s 543ms/step - loss: 1.1421 - accuracy: 0.6375 - val_loss: 1.3364 - val_accuracy: 0.5483
Epoch 5/8
344/344 [==============================] - 187s 543ms/step - loss: 0.8945 - accuracy: 0.7229 - val_loss: 1.2713 - val_accuracy: 0.5764
Epoch 6/8
344/344 [==============================] - 187s 543ms/step - loss: 0.6472 - accuracy: 0.8269 - val_loss: 1.4027 - val_accuracy: 0.5653
Epoch 7/8
344/344 [==============================] - 187s 543ms/ste

In [17]:
valid = learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.55      0.58      0.57       356
          10       0.00      0.00      0.00        10
          11       0.00      0.00      0.00        22
          12       0.25      0.02      0.04        48
          13       0.50      0.60      0.55       207
          14       0.20      0.23      0.21        13
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         3
          17       0.85      0.93      0.89       247
          18       0.00      0.00      0.00         4
          19       0.60      0.38      0.46        16
           2       0.00      0.00      0.00         5
          20       0.00      0.00      0.00         3
           3       0.49      0.55      0.52       273
           4       0.00      0.00      0.00         1
           5       0.27      0.20      0.23        15
           6       0.64    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
